In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

c:\anaconda3\envs\std\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\anaconda3\envs\std\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
c:\anaconda3\envs\std\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
INPUT_DIM = 5

X, y = make_classification(n_samples=1000, 
                           n_features=INPUT_DIM, 
                           n_informative=2, 
                           n_redundant=0,
                           n_clusters_per_class=1, 
                           random_state=42)

# print(X)
# print(y)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("X_train 데이터 개수 :", len(X_train))
print("y_train 데이터 개수 :", len(y_train))
print("X_test 데이터 개수 :", len(X_test))
print("y_test 데이터 개수 :", len(y_test))

X_train 데이터 개수 : 700
y_train 데이터 개수 : 700
X_test 데이터 개수 : 300
y_test 데이터 개수 : 300


In [4]:

# setting device on GPU if available, else CPU

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [5]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [6]:
class LogisticRegression(nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_dim, 1)

    def forward(self, x):
        out = self.linear(x)
        out = torch.sigmoid(out)
        return out

model = LogisticRegression(input_dim=INPUT_DIM).to(device)

In [7]:
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# BCELoss() -> 0 과 1 분류 이진 분류라서


In [8]:
num_epochs = 100

for epoch in range(num_epochs):
    for inputs, targets in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets.unsqueeze(1))
        loss.backward()
        optimizer.step()
    if epoch % 10 == 0:
        print(f'Epoch: {epoch+1} / {num_epochs}, Loss: {loss.item():.4f}')

Epoch: 1 / 100, Loss: 0.8378
Epoch: 11 / 100, Loss: 0.4113
Epoch: 21 / 100, Loss: 0.4089
Epoch: 31 / 100, Loss: 0.3012
Epoch: 41 / 100, Loss: 0.3273
Epoch: 51 / 100, Loss: 0.3044
Epoch: 61 / 100, Loss: 0.2820
Epoch: 71 / 100, Loss: 0.2182
Epoch: 81 / 100, Loss: 0.2303
Epoch: 91 / 100, Loss: 0.2383


In [12]:
# 모델 평가
threshold = 0.5

with torch.no_grad():
    correct = 0
    total = 0
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)

        pred = outputs.data
        pred[pred >= threshold] = 1
        pred[pred < threshold] = 0
        
        pred = pred.squeeze(dim=1)
        total += targets.size(0)
        correct += (pred == targets).sum().item()
    
    print('Accuracy of the network on the test data: %d %%' % (100 * correct / total))



Accuracy of the network on the test data: 88 %
